In [1]:
train_path = "data/00--raw/football-field-detection.v15i.yolov5pytorch/train/"

IMG_PATH = f"{train_path}images/0a2d9b_2_3_png.rf.2b39030ff9f2e93a34aa9ca69abbd77c.jpg"
LBL_PATH = f"{train_path}/labels/0a2d9b_2_3_png.rf.2b39030ff9f2e93a34aa9ca69abbd77c.txt"

# IMG_PATH = f"{train_path}images/0a2d9b_6_11_png.rf.2cfd6b6dad39a0f39eee5eb3d729823f.jpg"
# LBL_PATH = f"{train_path}/labels/0a2d9b_6_11_png.rf.2cfd6b6dad39a0f39eee5eb3d729823f.txt"


# right side
# IMG_PATH = f"{train_path}images/0a2d9b_7_14_png.rf.04beeed5de2d712614d10a1e75aae7b9.jpg"
# LBL_PATH = f"{train_path}/labels/0a2d9b_7_14_png.rf.04beeed5de2d712614d10a1e75aae7b9.txt"


In [2]:
import cv2
import numpy as np
from src.struct.shared_data   import SharedAnnotations
from src.visual.field         import FieldVisualizer, PitchConfig
from src.visual.visualizer    import Visualizer

# 1) Roboflow → your index mapping
rf2my = {
    0:   0,
    1:   7,
    2:  15, 
    3:  16,
    4:   8, 
    5:   2, 
    6:  17, 
    7:  18,
    8:   5,
    9:   9,
    10: 27,
    11: 28,
    12: 10,
    13: 23,
    14: 25,
    15: 26, 
    16: 24,
    17: 13,
    18: 29,
    19: 30,
    20: 14,
    21:  6,
    22: 21,
    23: 22,
    24:  1,
    25: 11,
    26: 19,
    27: 20,
    28: 12,# correct until now
    29:  3,
}



# class  xc  yc  w  h   x0  y0  v0   x1  y1  v1   x2  y2  v2 ... xN yN vN

# field     count       meaning                                     coordinate space
# class	    1	        always 0 (only one class: “football field”)	–
# xc yc w h	4	        YOLO bounding-box (we normally ignore it)	normalised (0-1)
# xi yi	    2 × (N+1)	position of landmark i	                    normalised (0-1)
# vi	    1 × (N+1)	visibility flag (0 = not labelled / occluded, 1 = labelled)


# 2) load one example
IMG = IMG_PATH
LBL = LBL_PATH

img = cv2.imread(IMG)
h, w = img.shape[:2]

# 3) parse Roboflow keypoints
vals = list(map(float, open(LBL).read().split()))
kp   = np.array(vals[5:], dtype=np.float32).reshape(-1,3)  # (num_pts, 3)

# 4) init shared‐data and field
shared    = SharedAnnotations()
fv        = FieldVisualizer(PitchConfig(draw_points=True))
model_pts = fv.get_hardcoded_model_points()  # shape (M,2)

# 7) set up the Visualizer
class DummyProcess:
    def __init__(self, sd):      self.shared_data = sd
    def is_done(self):           return True
    def on_mouse_click(self, x,y): pass

viz = Visualizer(
    field_config = PitchConfig(linewidth=2, draw_points=True),
    frame        = img,             # initial video frame
    class_names  = {}, 
    process      = DummyProcess(shared)
)

# prime the video side
viz.video_visualizer.update(type("VF",(object,),{
    "frame_id":0, "timestamp":0.0,
    "image":img,
    "detections":[]
})())

# 5) build correspondences
pts_img   = []
pts_model = []
for j, (xn,yn,vis) in enumerate(kp):
    if vis <= 0 or j not in rf2my:
        continue
    my_idx = rf2my[j]

    # image‐space pixel
    x_px, y_px = float(xn*w), float(yn*h)
    shared.captured_video_pts.append((x_px, y_px))
    pts_img.append([x_px, y_px])

    # model‐space metre
    mx, my = model_pts[my_idx]
    shared.ground_truth_pts.append((mx, my))
    pts_model.append([mx, my])

pts_img   = np.array(pts_img,   dtype=np.float32)
pts_model = np.array(pts_model, dtype=np.float32)

if len(pts_img) > 4:
    # 6) fit homography H: model→image
    H, _                 = cv2.findHomography(pts_model, pts_img, cv2.RANSAC, 3.0)
    shared.H_video2field = H

    # ─── EXPERIMENT: random image points → project to model ────────────────────
    num_rand = 12
    rand_px  = np.column_stack([
        np.random.uniform(0, w, num_rand),
        np.random.uniform(0, h, num_rand),
    ])

    # draw these as blue dots on the video
    shared.sampled_video_pts = [(float(x), float(y)) for x,y in rand_px]

    # now project them into model space via H_inv
    H_inv = np.linalg.inv(shared.H_video2field)
    rand_model = viz.transform_points(rand_px, H_inv)

    # draw these as _model‐space_ blue dots on the field
    shared.projected_detection_model_pts = [
        (float(mx), float(my)) for mx,my in rand_model
    ]

# ─── 8) annotate & render ─────────────────────────────────────────────────
viz.video_visualizer.clear_annotations()
viz.field_visualizer.clear_annotations()
viz._annotate_frames(
    viz.video_visualizer.frame,
    viz.field_visualizer.frame
)
out = viz.generate_combined_view()

# ─── 9) display ────────────────────────────────────────────────────────────
display_scale = 0.8  # e.g. 50% of original size
out_resized = cv2.resize(out, None, fx=display_scale, fy=display_scale, interpolation=cv2.INTER_AREA)

cv2.imshow("Random Projection Test", out_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
from pathlib import Path
from src.visual.field import FieldVisualizer, PitchConfig

# Roboflow→your‐model index mapping
RF2MY = {
    0:   0,
    1:   7,
    2:  15, 
    3:  16,
    4:   8, 
    5:   2, 
    6:  17, 
    7:  18,
    8:   5,
    9:   9,
    10: 27,
    11: 28,
    12: 10,
    13: 23,
    14: 25,
    15: 26, 
    16: 24,
    17: 13,
    18: 29,
    19: 30,
    20: 14,
    21:  6,
    22: 21,
    23: 22,
    24:  1,
    25: 11,
    26: 19,
    27: 20,
    28: 12,# correct until now
    29:  3,
}

import os
import cv2
import numpy as np
from pathlib import Path
from src.visual.field import FieldVisualizer, PitchConfig

# Roboflow→your‐model index mapping
RF2MY = {
    0:0, 1:7,  2:15, 3:16, 4:8,  5:2,  6:17, 7:18,
    8:5, 9:9,10:27,11:28,12:10,13:23,14:25,15:26,
   16:24,17:13,18:29,19:30,20:14,21:6, 22:21,23:22,
   24:1,25:11,26:19,27:20,28:12,29:3
}

def compute_homography_pixel2field(label_path: str,
                                   model_pts: np.ndarray,
                                   pitch_cfg: PitchConfig,
                                   img_shape: tuple) -> np.ndarray:
    """
    Reads a YOLO‐kp label and returns H mapping (x_px, y_px, 1) →
    (X_m, Y_m, w) so that [X_m, Y_m] are in metres [0..length],[0..width].
    """
    img_h, img_w = img_shape[:2]
    vals = list(map(float, open(label_path).read().split()))
    kp   = np.array(vals[5:], dtype=np.float32).reshape(-1,3)

    pts_img   = []
    pts_model = []
    for j,(xn,yn,vis) in enumerate(kp):
        if vis <= 0 or j not in RF2MY:
            continue
        # pixel coordinates
        x_px, y_px = float(xn * img_w), float(yn * img_h)
        pts_img.append([x_px, y_px])
        # meter coordinates
        mx, my = model_pts[RF2MY[j]]
        pts_model.append([mx, my])

    pts_img   = np.array(pts_img,   dtype=np.float32)
    pts_model = np.array(pts_model, dtype=np.float32)

    if len(pts_img) < 4:
        return None

    # **DIRECT** mapping: pixels → metres
    H_px2m, mask = cv2.findHomography(pts_img, pts_model,
                                      cv2.RANSAC, ransacReprojThreshold=3.0)
    if H_px2m is None:
        return None

    # normalize so H[2,2]==1
    H_px2m /= H_px2m[2,2]
    return H_px2m

def clean_dataset_pixel2field(input_dir: str, output_dir: str):
    """
    Flattens train/valid/test into output_dir:
      frame_000001.png
      frame_000001_H.npy   (H_pixel2field)
    """
    input_dir  = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    # prepare model points once
    fv        = FieldVisualizer(PitchConfig())
    model_pts = fv.get_hardcoded_model_points()

    idx = 0
    for split in ("train","valid","test"):
        imgs = sorted((input_dir/split/"images").glob("*"))
        for img_path in imgs:
            lbl_path = input_dir/split/"labels"/f"{img_path.stem}.txt"
            if not lbl_path.exists():
                continue

            img = cv2.imread(str(img_path))
            if img is None:
                continue

            H = compute_homography_pixel2field(
                    str(lbl_path), model_pts, img.shape)
            if H is None:
                print(f"⚠ skipping {img_path.name}: <4 pts or bad H>")
                continue

            idx += 1
            out_img = output_dir/f"frame_{idx:06d}.png"
            out_H   = output_dir/f"frame_{idx:06d}_H.npy"

            cv2.imwrite(str(out_img), img)
            np.save(str(out_H), H)

    print(f"✅ Wrote {idx} frames + H_pixel2field to {output_dir}")



clean_dataset_pixel2field(
    "data/00--raw/football-field-detection.v15i.yolov5pytorch",
    "data/01--clean/roboflow-pixel2field",
)

TypeError: compute_homography_pixel2field() missing 1 required positional argument: 'img_shape'

In [4]:
import os
import cv2
import numpy as np
from pathlib import Path
from src.visual.field import FieldVisualizer, PitchConfig

# Roboflow→your‐model index mapping
RF2MY = {
    0:   0,
    1:   7,
    2:  15, 
    3:  16,
    4:   8, 
    5:   2, 
    6:  17, 
    7:  18,
    8:   5,
    9:   9,
    10: 27,
    11: 28,
    12: 10,
    13: 23,
    14: 25,
    15: 26, 
    16: 24,
    17: 13,
    18: 29,
    19: 30,
    20: 14,
    21:  6,
    22: 21,
    23: 22,
    24:  1,
    25: 11,
    26: 19,
    27: 20,
    28: 12,# correct until now
    29:  3,
}


def compute_homography_normalized(
    label_path: str,
    model_pts: np.ndarray,
    img_shape: tuple,
    pitch_cfg: PitchConfig
) -> np.ndarray:
    """
    Reads a YOLO‐kp label and returns H_norm mapping
      [x_px/W, y_px/H, 1]^T → [X_m/L, Y_m/W, 1]^T
    where L=length, W=width from pitch_cfg.
    Both source and destination coordinates live in [0,1].
    """
    H_px, W_px = img_shape[:2]
    L, W       = pitch_cfg.length, pitch_cfg.width

    vals = list(map(float, open(label_path).read().split()))
    kp   = np.array(vals[5:], dtype=np.float32).reshape(-1, 3)

    src_norm = []
    dst_norm = []
    for j, (xn, yn, vis) in enumerate(kp):
        if vis <= 0 or j not in RF2MY:
            continue
        # normalized image coords
        src_norm.append([xn, yn])
        # normalized field coords
        mx, my = model_pts[RF2MY[j]]
        dst_norm.append([mx / L, my / W])

    src_norm = np.array(src_norm, dtype=np.float32)
    dst_norm = np.array(dst_norm, dtype=np.float32)
    if len(src_norm) < 4:
        return None

    # solve in normalized space with a small reproj threshold
    Hn, mask = cv2.findHomography(
        src_norm, dst_norm,
        method=cv2.RANSAC,
        ransacReprojThreshold=1e-3
    )
    if Hn is None:
        return None

    # normalize so Hn[2,2] == 1
    Hn /= Hn[2,2]
    return Hn.astype(np.float32)


def clean_dataset_normalized(input_dir: str, output_dir: str):
    """
    Walk through train/valid/test under input_dir, compute normalized homography,
    and write into output_dir:
      frame_000001.png
      frame_000001_Hnorm.npy
    """
    input_dir  = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # prepare model points once
    fv        = FieldVisualizer(PitchConfig())
    model_pts = fv.get_hardcoded_model_points()
    pitch_cfg = fv.config

    idx = 0
    for split in ("train", "valid", "test"):
        img_folder = input_dir/split/"images"
        lbl_folder = input_dir/split/"labels"
        for img_path in sorted(img_folder.glob("*")):
            lbl_path = lbl_folder/f"{img_path.stem}.txt"
            if not lbl_path.exists():
                continue

            img = cv2.imread(str(img_path))
            if img is None:
                continue

            Hn = compute_homography_normalized(
                str(lbl_path),
                model_pts,
                img.shape,
                pitch_cfg
            )
            if Hn is None:
                print(f"⚠ skipping {img_path.name}: <4 pts or no H>")
                continue

            idx += 1
            out_img = output_dir/f"frame_{idx:06d}.png"
            out_H   = output_dir/f"frame_{idx:06d}_H.npy"

            cv2.imwrite(str(out_img), img)
            np.save(str(out_H), Hn)

    print(f"✅ Finished. Wrote {idx} frames + normalized homographies to {output_dir}")


clean_dataset_normalized(
    "data/00--raw/football-field-detection.v15i.yolov5pytorch",
    "data/01--clean/roboflow",
)


⚠ skipping 08fd33_0_9_png.rf.26e5e8716ff2eef771bbc7b876897685.jpg: <4 pts or no H>
⚠ skipping 08fd33_6_2_png.rf.daab0ee8baa0b5090fe877bb30a00acf.jpg: <4 pts or no H>
⚠ skipping 08fd33_6_4_png.rf.9c00c3899e6a1a4a8511162aea8ce223.jpg: <4 pts or no H>
⚠ skipping 08fd33_3_5_png.rf.e7272099dcba3f202ae4da1010b8d7e2.jpg: <4 pts or no H>
⚠ skipping 54745b_1_3_png.rf.77f1b370d7946cfc4b5d689bf13ad67d.jpg: <4 pts or no H>
✅ Finished. Wrote 312 frames + normalized homographies to data/01--clean/roboflow
